## Loss Functions

In this assignment, we will learn about loss functions. We will use a create a neural network and measure the model's performance using different loss functions.

In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense

pd.set_option('display.max_rows', 100)

In [2]:
housing = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/housing.csv')

In [3]:
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


We will use the dataset above to predict housing prices using various features about each house. Our first step is to check for missing data. 

In [16]:
# Answer below:
missing_pct = (housing.isnull().sum() / housing.isnull().count()) #> 0.3
missing_pct.sort_values(ascending=False).head(6)

PoolQC         0.995205
MiscFeature    0.963014
Alley          0.937671
Fence          0.807534
FireplaceQu    0.472603
LotFrontage    0.177397
dtype: float64

Remove columns that contain more than 30% of missing data. After removing those columns, remove the rows that contain at least one observation that is missing.

In [20]:
# Answer below:
col_thresh = len(housing) * .7
print(housing.shape)
clean_housing = housing.dropna(thresh=col_thresh, axis=1).dropna(axis=0)
print(clean_housing.shape)

(1460, 81)
(1094, 76)


There are some categorical variables that contain numeric data and some that do not. Print the type of each column to first see whether there is an issue with misclassification of column type.

In [53]:
clean_housing.describe(include='all')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1094.000000,1094.000000,1094,1094.000000,1094.000000,1094,1094,1094,1094,1094,1094,1094,1094,1094,1094,1094,1094.000000,1094.000000,1094.000000,1094.000000,1094,1094,1094,1094,1094,1094.000000,1094,1094,1094,1094,1094,1094,1094,1094.000000,1094,1094.000000,1094.000000,1094.000000,1094,1094,1094,1094,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094,1094.000000,1094,1094.000000,1094,1094.000000,1094,1094.000000,1094.000000,1094,1094,1094,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094,1094,1094.000000
unique,NaN,NaN,5,NaN,NaN,2,4,4,1,5,3,25,9,6,5,8,NaN,NaN,NaN,NaN,5,7,14,16,4,NaN,4,4,5,4,4,4,6,NaN,6,NaN,NaN,NaN,4,5,2,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,6,NaN,6,NaN,3,NaN,NaN,5,5,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,NaN,NaN,NaN,NaN,Gable,CompShg,VinylSd,VinylSd,None,NaN,TA,TA,PConc,TA,TA,No,Unf,NaN,Unf,NaN,NaN,NaN,GasA,Ex,Y,SBrkr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TA,NaN,Typ,NaN,Attchd,NaN,Unf,NaN,NaN,TA,TA,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,850,NaN,NaN,1090,760,991,1094,830,1045,173,950,1082,925,540,NaN,NaN,NaN,NaN,843,1078,421,412,639,NaN,646,973,518,486,1006,734,343,NaN,972,NaN,NaN,NaN,1075,594,1036,1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,528,NaN,1024,NaN,680,NaN,485,NaN,NaN,1031,1050,1023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,928,880,NaN
mean,727.375686,56.128885,NaN,70.759598,10132.346435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.247715,5.575868,1972.412249,1985.915905,NaN,NaN,NaN,NaN,NaN,109.855576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,448.191956,NaN,45.252285,606.117002,1099.561243,NaN,NaN,NaN,NaN,1173.809872,356.536563,4.680987,1535.027422,0.423218,0.056673,1.577697,0.389397,2.861974,1.033821,NaN,6.570384,NaN,0.610603,NaN,1978.565814,NaN,1.879342,503.760512,NaN,NaN,NaN,94.341865,46.946984,22.053016,3.266910,16.498172,3.007313,23.550274,6.335466,2007.786106,NaN,NaN,187033.263254
std,420.955488,41.976345,NaN,24.508859,8212.249621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.366797,1.066500,31.189752,20.930772,NaN,NaN,NaN,NaN,NaN,190.667459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,468.728095,NaN,159.075003,445.834636,415.851262,NaN,NaN,NaN,NaN,387.677463,439.260130,42.099821,526.124028,0.514253,0.235244,0.550219,0.500793,0.764107,0.185841,NaN,1.584486,NaN,0.633118,NaN,25.934444,NaN,0.658586,192.261314,NaN,NaN,NaN,122.624615,64.820019,61.570502,29.655973,58.455303,40.713175,167.135237,2.694558,1.334307,NaN,NaN,83165.332151
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,1880.000000,1950.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,105.000000,NaN,NaN,NaN,NaN,438.000000,0.000000,0.000000,438.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,3.000000,NaN,0.000000,NaN,1900.000000,NaN,1.000000,160.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,NaN,NaN,35311.000000
25%,366.500000,20.000000,

We see that month sold and year sold are not variables that describe a feature of the house. While they do have relevance if we create a model containing a time series element, we will not include them here. Drop these columns. Also, remove the id column

In [55]:
# Answer below
clean_housing_df = clean_housing.drop(columns=['Id', 'MoSold', 'YrSold'])

Using the information about the column types, identify all the variables that will be converted into dummy variables. Include at least one numeric variable that you think should be converted as well.

In [61]:
to_be_encoded = ['OverallQual', 'OverallCond']

Convert the columns you selected above into dummy variables

In [62]:
# Answer below:
print(clean_housing_df.shape)
num_dummies = pd.get_dummies(clean_housing_df, columns=to_be_encoded, drop_first=True)
print(num_dummies.shape)
dummied_df = pd.get_dummies(num_dummies,drop_first=True)
print(dummied_df.shape)

(1094, 73)
(1094, 86)
(1094, 233)


Split the data into train and test with 20% of data in test.

In [63]:
# Answer below
from sklearn.model_selection import train_test_split

X = dummied_df.drop(columns=['SalePrice'])
y = dummied_df.SalePrice

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [65]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Create a model with 5 layers. The first layer should be a dense layer that takes in the input, the last layer should be of size 1. You determine the remaining layer sizes.

Use a linear activation for the output layer.

In [70]:
# Answer below
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_ANN_model(loss, optimizer, metrics):
  model = Sequential()

  model.add(Dense(80, input_dim=X_train_scaled.shape[1], activation='relu'))
  model.add(Dense(60, activation='relu'))
  model.add(Dense(40, activation='relu'))
  model.add(Dense(20, activation='relu'))
  model.add(Dense(1, activation='linear'))

  model.compile(loss=loss, optimizer= optimizer, metrics=metrics)
  model.fit(X_train, y_train, epochs=200, batch_size=100, validation_data=(X_test_scaled, y_test), verbose=0)
  print(model.evaluate(X_test, y_test)[1])

Compile the model with the RMSprop optimizer and mean square error loss. Use the MSE as a metric. Set batch size to 100 and epochs to 200. Fit the model and report the results.

In [71]:
# Answer below:
build_ANN_model('mse', 'RMSprop', 'mse')

7/7 [==============================] - 0s 2ms/step - loss: 1732889600.0000 - mse: 1732889600.0000
1732889600.0


Next, do the same but with mean absolute error loss. Use both MSE and MAE as metrics. Compare the results.

In [72]:
# Answer below:
build_ANN_model('mae', 'RMSprop', ['mse', 'mae'])

7/7 [==============================] - 0s 2ms/step - loss: 26656.0508 - mse: 1573649280.0000 - mae: 26656.0508
1573649280.0


Finally, try your model using mean squared logarithmic error. Compare the results.

In [73]:
# Answer below:
build_ANN_model('msle', 'RMSprop', ['mse', 'mae'])

7/7 [==============================] - 0s 2ms/step - loss: 0.0390 - mse: 1926609920.0000 - mae: 30288.1270
1926609920.0
